# Advanced Econometrics (JEM217)– Home Assignment 3 (WS 2025)



**Authors: Sebastian Pasz, Vojtěch Dohnal, Tomáš Gabriel**

**AI Usage**

ChatGPT-5 was used during the preparation of this assignment **solely for grammatical editing and minor code optimization**.  
Any sections where AI-assisted code optimization was applied are explicitly indicated within the notebook.

## Initial Preparation

In this section, we install and load all required R packages necessary for data simulation, estimation, and visualization.  
We also set a fixed random seed (`set.seed(123)`) to ensure that all simulated results are fully reproducible.


In [1]:

req_pkgs <- c("ggplot2","dplyr","quantreg","lmtest","sandwich","car","stargazer","knitr","glm2","gmm","msm","plm")
for (p in req_pkgs) {
  if (!requireNamespace(p, quietly = TRUE)) {
    install.packages(p, repos = "https://cloud.r-project.org")
  }
}
library(ggplot2)
library(dplyr)
library(quantreg)
library(lmtest)
library(sandwich)
library(car)
library(stargazer)
library(knitr)
library(glm2)
library(gmm)
library(msm)
library(plm)


set.seed(123) 


Warning message:
"package 'dplyr' was built under R version 4.4.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'quantreg' was built under R version 4.4.3"
Loading required package: SparseM

Warning message:
"package 'SparseM' was built under R version 4.4.3"
Warning message:
"package 'lmtest' was built under R version 4.4.3"
Loading required package: zoo

Warning message:
"package 'zoo' was built under R version 4.4.3"

Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Warning message:
"package 'sandwich' was built under R version 4.4.3"
Warning message:
"package 'car' was built under R version 4.4.3"
Loading required package: carData

Warning message:
"package 'carData' was built under R version 4.4.3"

Attaching package: 'car'


The following

## Problem 2.

First we download the dataset and analyze the original results of Arellano and Bond (1991).

In [11]:
### Original results of Arellano and Bond (1991)
data("EmplUK")
emp.gmm <- pgmm(
log(emp) ~ lag(log(emp), 1:2) + lag(log(wage), 0:1) + log(capital) +
lag(log(output), 0:1) | lag(log(emp), 2:99),
data = EmplUK, effect = "twoways", model = "twosteps"
)
summary(emp.gmm, robust = FALSE)

Twoways effects Two-steps model Difference GMM 

Call:
pgmm(formula = log(emp) ~ lag(log(emp), 1:2) + lag(log(wage), 
    0:1) + log(capital) + lag(log(output), 0:1) | lag(log(emp), 
    2:99), data = EmplUK, effect = "twoways", model = "twosteps")

Unbalanced Panel: n = 140, T = 7-9, N = 1031

Number of Observations Used: 611
Residuals:
      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-0.6190677 -0.0255683  0.0000000 -0.0001339  0.0332013  0.6410272 

Coefficients:
                        Estimate Std. Error  z-value  Pr(>|z|)    
lag(log(emp), 1:2)1     0.474151   0.085303   5.5584 2.722e-08 ***
lag(log(emp), 1:2)2    -0.052967   0.027284  -1.9413 0.0522200 .  
lag(log(wage), 0:1)0   -0.513205   0.049345 -10.4003 < 2.2e-16 ***
lag(log(wage), 0:1)1    0.224640   0.080063   2.8058 0.0050192 ** 
log(capital)            0.292723   0.039463   7.4177 1.191e-13 ***
lag(log(output), 0:1)0  0.609775   0.108524   5.6188 1.923e-08 ***
lag(log(output), 0:1)1 -0.446373   0.12481

### (a) 

In [ ]:
EmplUKp <- pdata.frame(EmplUK, index = c("firm", "year"))

FE <- plm(
  log(emp) ~ lag(log(wage), 0:1) + log(capital) + lag(log(output), 0:1),
  data   = EmplUKp,
  model  = "within",
  effect = "individual"   # firm fixed effects
)

summary(FE)

Oneway (individual) effect Within Model

Call:
plm(formula = log(emp) ~ lag(log(wage), 0:1) + log(capital) + 
    lag(log(output), 0:1), data = EmplUKp, effect = "individual", 
    model = "within")

Unbalanced Panel: n = 140, T = 6-8, N = 891

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-0.4988098 -0.0680434  0.0019928  0.0699134  0.4752333 

Coefficients:
                        Estimate Std. Error t-value  Pr(>|t|)    
lag(log(wage), 0:1)0   -0.534946   0.064348 -8.3133 4.395e-16 ***
lag(log(wage), 0:1)1    0.059360   0.060046  0.9886    0.3232    
log(capital)            0.557570   0.024931 22.3645 < 2.2e-16 ***
lag(log(output), 0:1)0  0.508685   0.090605  5.6143 2.784e-08 ***
lag(log(output), 0:1)1 -0.097916   0.104054 -0.9410    0.3470    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Total Sum of Squares:    32.324
Residual Sum of Squares: 11.545
R-Squared:      0.64282
Adj. R-Squared: 0.57388
F-statistic: 268.519 on 5 and 746 DF, p-v